<img src="http://xarray.pydata.org/en/stable/_static/dataset-diagram-logo.png" align="right" width="30%">

# Computation with Xarray

In this lesson, we discuss how to do scientific computations with xarray
objects. Our learning goals are as follows. By the end of the lesson, we will be
able to:

- Apply basic arithmetic and numpy functions to xarray DataArrays / Dataset.
- Use Xarray's label-aware reduction operations (e.g. `mean`, `sum`) weighted
  reductions.
- Apply arbitrary functions to Xarray data via `apply_ufunc`.
- Use Xarray's broadcasting to compute on arrays of different dimensionality.
- Perform "split / apply / combine" workflows in Xarray using `groupby`,
  including
  - reductions within groups
  - transformations on groups
- Use the `resample`, `rolling` and `coarsen` functions to manipulate data.


In [ ]:
import numpy as np
import xarray as xr
from matplotlib import pyplot as plt

## Example Dataset

First we load a dataset. We will use the
[NOAA Extended Reconstructed Sea Surface Temperature (ERSST) v5](https://www.ncdc.noaa.gov/data-access/marineocean-data/extended-reconstructed-sea-surface-temperature-ersst-v5)
product, a widely used and trusted gridded compilation of of historical data
going back to 1854.

Since the data is provided via an
[OPeNDAP](https://en.wikipedia.org/wiki/OPeNDAP) server, we can load it directly
without downloading anything:


In [ ]:
### NOTE: If hundreds of people connect to this server at once and download the same dataset,
###       things might not go so well! Recommended to use the Google Cloud copy instead.

# url = "http://www.esrl.noaa.gov/psd/thredds/dodsC/Datasets/noaa.ersst.v5/sst.mnmean.nc"
# # drop an unnecessary variable which complicates some operations
# ds = xr.open_dataset(url, drop_variables=["time_bnds"])
# # will take a minute or two to complete
# ds = ds.sel(time=slice("1960", "2018")).load()
# ds

In [ ]:
import gcsfs

fs = gcsfs.GCSFileSystem(token="anon")
ds = xr.open_zarr(
    fs.get_mapper("gs://pangeo-noaa-ncei/noaa.ersst.v5.zarr"), consolidated=True
).load()
ds

Let's do some basic visualizations of the data, just to make sure it looks
reasonable.


In [ ]:
ds.sst[0].plot(vmin=-2, vmax=30)

## Basic Arithmetic

Xarray dataarrays and datasets work seamlessly with arithmetic operators and
numpy array functions.

For example, imagine we want to convert the temperature (given in Celsius) to
Kelvin:


In [ ]:
sst_kelvin = ds.sst + 273.15
sst_kelvin

The dimensions and coordinates were preseved following the operation.

<div class="alert alert-warning">
    <strong>Warning:</strong> Although many xarray datasets have a <code>units</code> attribute, which is used in plotting,
    Xarray does not inherently understand units. However, work is underway to integrate xarray
    with <a href="https://pint.readthedocs.io/en/0.12/">pint</a>, which will provide full unit-aware operations.
</div>

We can apply more complex functions, including numpy ufuncs, to Xarray objects.
Imagine we wanted to compute the following expression as a function of SST
($\Theta$) in Kelvin:

$$ f(\Theta) =  0.5 \ln(\Theta^2) $$


In [ ]:
f = 0.5 * np.log(sst_kelvin**2)
f

## Applying Aribtrary Functions

It's awesome that we can call `np.log(ds)` and have it "just work". However, not
all third party libraries work this way.

In this example, we will use functions from the
[Gibbs Seawater Toolkit](https://teos-10.github.io/GSW-Python/), a package for
the thermodynamics of seawater. This package provides ufuncs that operate on
numpy arrays.


In [ ]:
import gsw

# an example function
# http://www.teos-10.org/pubs/gsw/html/gsw_t90_from_t68.html
?gsw.t90_from_t68

In [ ]:
gsw.t90_from_t68(ds.sst)  # -> returns a numpy array

It would be nice to keep our dimensions and coordinates. We can accomplish this
with `xr.apply_ufunc`.


In [ ]:
xr.apply_ufunc(gsw.t90_from_t68, ds.sst)

<div class="alert alert-info">
    <strong>Note:</strong> <code>apply_ufunc</code> is a powerful and mysterious function.
    It has many options for doing more complicated things.
    Unfortunately, we don't have time to go into more depth here.
    Please consult the [Xarray docs](http://xarray.pydata.org/en/latest/generated/xarray.apply_ufunc.html) for more details.
</div>


## Reductions

Just like in numpy, we can reduce xarray DataArrays along any number of axes:


In [ ]:
sst = ds.sst
sst.mean(axis=0)

In [ ]:
sst.mean(axis=(1, 2))

In [ ]:
sst.mean()

However, rather than performing reductions on axes (as in numpy), we can perform
them on dimensions. This turns out to be a huge convenience, particularly in
complex calculations when you can't easily remember which axis corresponds to
which dimension:


In [ ]:
sst.mean(dim="time")

All of the standard numpy reductions (e.g. `min`, `max`, `sum`, `std`, etc.) are
available


### Exercise

Take the mean of `sst` in both longitude and latitude. Make a simple timeseries
plot:


In [ ]:
# your code here

## Broadcasting

Broadcasting refers to the alignmed of arrays with different numbers of
dimensions. Numpy's broadcasting rules, based on array shape, can sometimes be
difficult to understand and remember. Xarray does brodcasting by dimension name,
rather than array shape. This is a huge convenience.

Let's now create two arrays with some dimensions in common. For this example, we
will create a "weights" array proportional to cosine of latitude. Modulo a
normalization, this is the correct area-weighting factor for data on a regular
lat-lon grid.


In [ ]:
weights = np.cos(np.deg2rad(ds.lat))
weights.dims

If we multiply this by SST, it "just works," and the arrays are broadcasted
properly:


In [ ]:
(ds.sst * weights).dims

<div class="alert alert-warning">
    <strong>Warning:</strong> If the arrays being broadcasted share a dimension name, but have different coordinates,
    they will first be aligned using Xarray's default align settings (including filling missing values with NaNs).
    If that's not what you want, it's best to call <code>align</code> explicitly before broadcasting.
</div>


## Weighted Reductions

We could imagine computing the weighted spatial mean of SST manually, like this:


In [ ]:
sst_mean = (ds.sst * weights).sum(dim=("lon", "lat")) / weights.sum(dim="lat")
sst_mean.plot()
plt.title("This is wrong!")

That would be wrong, however, because the denominator (`weights.sum(dim='lat')`)
needs to be expanded to include the `lon` dimension and modified to account for
the missing values (land points).

In general, weighted reductions on multidimensional arrays are complicated. To
make it a bit easier, Xarray provides a mechanism for weighted reductions. It
does this by creating a special intermediate `DataArrayWeighted` object, to
which different reduction operations can applied.


In [ ]:
sst_weighted = ds.sst.weighted(weights)
sst_weighted

In [ ]:
sst_weighted.mean(dim=("lon", "lat")).plot()
plt.title("Correct Global Mean SST")

## Groupby

Xarray copies Pandas' very useful groupby functionality, enabling the "split /
apply / combine" workflow on xarray DataArrays and Datasets.

To provide a physically motivated example, let's examine a timeseries of SST at
a single point.


In [ ]:
ds.sst.sel(lon=300, lat=50).plot()

As we can see from the plot, the timeseries at any one point is totally
dominated by the seasonal cycle. We would like to remove this seasonal cycle
(called the "climatology") in order to better see the long-term variaitions in
temperature. We can accomplish this using **groupby**.

Before moving forward, w note that xarray correctly parsed the time index,
resulting in a Pandas datetime index on the time dimension.


In [ ]:
ds.time

The syntax of Xarray's groupby is almost identical to Pandas.


In [ ]:
?ds.groupby

### Split Step

The most important argument is `group`: this defines the unique values we will
us to "split" the data for grouped analysis. We can pass either a DataArray or a
name of a variable in the dataset. Lets first use a DataArray. Just like with
Pandas, we can use the time index to extract specific components of dates and
times. Xarray uses a special syntax for this `.dt`, called the
`DatetimeAccessor`.


In [ ]:
ds.time.dt

In [ ]:
ds.time.dt.month

ds.time.dt.year


We can use these arrays in a groupby operation:


In [ ]:
gb = ds.groupby(ds.time.dt.month)
gb

Xarray also offers a more concise syntax when the variable you're grouping on is
already present in the dataset. This is identical to the previous line:


In [ ]:
gb = ds.groupby("time.month")
gb

Now that the data are split, we can manually iterate over the group. The
iterator returns the key (group name) and the value (the actual dataset
corresponding to that group) for each group.


In [ ]:
for group_name, group_ds in gb:
    # stop iterating after the first loop
    break
print(group_name)
group_ds

### Apply & Combine

Now that we have groups defined, it's time to "apply" a calculation to the
group. Like in Pandas, these calculations can either be:

- _aggregation_: reduces the size of the group
- _transformation_: preserves the group's full size

At then end of the apply step, xarray will automatically combine the aggregated
/ transformed groups back into a single object.

The most fundamental way to apply is with the `.map` method.


In [ ]:
?gb.map

#### Aggregations

`.map` accepts as its argument a function that expects and return xarray
objects. We define a custom function. This function takes a single argument--the
group dataset--and returns a new dataset to be combined:


In [ ]:
def time_mean(a):
    return a.mean(dim="time")


gb.map(time_mean)

Like Pandas, xarray's groupby object has many built-in aggregation operations
(e.g. `mean`, `min`, `max`, `std`, etc):


In [ ]:
# this does the same thing as the previous cell
ds_mm = gb.mean(dim="time")
ds_mm

So we did what we wanted to do: calculate the climatology at every point in the
dataset. Let's look at the data a bit.

_Climatlogy at a specific point in the North Atlantic_


In [ ]:
ds_mm.sst.sel(lon=300, lat=50).plot()

_Zonal Mean Climatolgoy_


In [ ]:
ds_mm.sst.mean(dim="lon").plot.contourf(x="month", levels=12, vmin=-2, vmax=30)

_Difference between January and July Climatology_


In [ ]:
(ds_mm.sst.sel(month=1) - ds_mm.sst.sel(month=7)).plot(vmax=10)

#### Transformations

Now we want to _remove_ this climatology from the dataset, to examine the
residual, called the _anomaly_, which is the interesting part from a climate
perspective. Removing the seasonal climatology is a perfect example of a
transformation: it operates over a group, but doesn't change the size of the
dataset. Here is one way to code it


In [ ]:
def remove_time_mean(x):
    return x - x.mean(dim="time")


ds_anom = ds.groupby("time.month").map(remove_time_mean)
ds_anom

Xarray makes these sorts of transformations easy by supporting _groupby
arithmetic_. This concept is easiest explained with an example:


In [ ]:
gb = ds.groupby("time.month")
ds_anom = gb - gb.mean(dim="time")
ds_anom

Now we can view the climate signal without the overwhelming influence of the
seasonal cycle.

_Timeseries at a single point in the North Atlantic_


In [ ]:
ds_anom.sst.sel(lon=300, lat=50).plot()

_Difference between Jan. 1 2018 and Jan. 1 1960_


In [ ]:
(ds_anom.sel(time="2018-01-01") - ds_anom.sel(time="1960-01-01")).sst.plot()

## Grouby-Related: Resample, Rolling, Coarsen

Resample in xarray is nearly identical to Pandas. It is effectively a group-by
operation, and uses the same basic syntax. It can be applied only to time-index
dimensions. Here we compute the five-year mean.


In [ ]:
resample_obj = ds_anom.resample(time="5Y")
resample_obj

In [ ]:
ds_anom_resample = resample_obj.mean(dim="time")
ds_anom_resample

In [ ]:
ds_anom.sst.sel(lon=300, lat=50).plot()
ds_anom_resample.sst.sel(lon=300, lat=50).plot(marker="o")

<div class="alert alert-info">
    <strong>Note:</strong> <code>resample</code> only works with proper datetime indexes.
</div>

Rolling is also similar to pandas, but can be applied along any dimension. It
works with logical coordinates.


In [ ]:
ds_anom_rolling = ds_anom.rolling(time=12, center=True).mean()
ds_anom_rolling

In [ ]:
ds_anom.sst.sel(lon=300, lat=50).plot(label="monthly anom")
ds_anom_resample.sst.sel(lon=300, lat=50).plot(marker="o", label="5 year resample")
ds_anom_rolling.sst.sel(lon=300, lat=50).plot(label="12 month rolling mean")
plt.legend()

`coarsen` does something similar to `resample`, but without being aware of time.
It operates on logical coordinates only but can work on multiple dimensions at a
time.


In [ ]:
ds_anom_coarsen_time = ds_anom.coarsen(time=12).mean()

ds_anom_rolling.sst.sel(lon=300, lat=50).plot(label="12 month rolling mean")
ds_anom_coarsen_time.sst.sel(lon=300, lat=50).plot(marker="^", label="12 item coarsen")
plt.legend()

In [ ]:
# We expect an error here
ds_anom_coarsen_space = ds_anom.coarsen(lon=4, lat=4).mean()

In [ ]:
ds_anom_coarsen_space = ds_anom.isel(lat=slice(0, -1)).coarsen(lon=4, lat=4).mean()
ds_anom_coarsen_space

In [ ]:
ds_anom_coarsen_space.sst.isel(time=0).plot()

## Exercise

Load the following "basin mask" dataset, and use it to take a weighted average
of SST in each ocean basin. Figure out which ocean basins are the warmest and
coldest.

**Hint:** you will first need to align this dataset with the SST dataset. Use
what you learned in the "indexing and alignment" lesson.


In [ ]:
basin = xr.open_dataset(
    "http://iridl.ldeo.columbia.edu/SOURCES/.NOAA/.NODC/.WOA09/.Masks/.basin/dods"
)
basin